<a href="https://colab.research.google.com/github/dsandeep0138/HackerEarth-Challenges/blob/master/flower_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q kaggle

In [6]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dsandeep97","key":"91ed93d9f57d24d723707d120074f706"}'}

In [0]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
 !chmod 600 ~/.kaggle/kaggle.json

In [9]:
!kaggle datasets download -d rednivrug/flower-recognition-he

100% 905M/905M [00:16<00:00, 51.5MB/s]
100% 905M/905M [00:16<00:00, 56.9MB/s]


In [0]:
!unzip flower-recognition-he.zip

In [0]:
import gc, cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau

from keras.preprocessing.image import ImageDataGenerator

In [0]:
df = pd.read_csv('data/train.csv', dtype=str)
testdf = pd.read_csv('data/test.csv', dtype=str)

In [0]:
df['image_id'] = df['image_id'] + ".jpg"
testdf['image_id'] = testdf['image_id'] + ".jpg"

In [20]:
datagen=ImageDataGenerator(rescale=1./255.,
                           rotation_range=20,
                           zoom_range = 0.20,
                           width_shift_range=0.2, 
                           height_shift_range=0.2)

train_generator = datagen.flow_from_dataframe(
dataframe=df,
directory="./data/train/",
x_col="image_id",
y_col="category",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(128, 128))

Found 18540 validated image filenames belonging to 102 classes.


In [21]:
test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(
dataframe=testdf,
directory="./data/test/",
x_col="image_id",
y_col=None,
batch_size=1,
seed=42,
shuffle=False,
class_mode=None,
target_size=(128, 128))

Found 2009 validated image filenames.


In [22]:
model = tf.keras.Sequential()
model.add(layers.Conv2D(32, kernel_size=(5, 5),
                 activation='relu',
                 padding='same',
                 input_shape=(128, 128, 3)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(32, (5, 5), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(layers.Dropout(0.2))
  
model.add(layers.Flatten())  
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))
  
model.add(layers.Dense(512, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))
  
model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))

model.add(layers.Dense(102, activation='softmax'))
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_35 (Conv2D)           (None, 128, 128, 32)      2432      
_________________________________________________________________
batch_normalization_51 (Batc (None, 128, 128, 32)      128       
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 124, 124, 32)      25632     
_________________________________________________________________
batch_normalization_52 (Batc (None, 124, 124, 32)      128       
_________________________________________________________________
dropout_37 (Dropout)         (None, 124, 124, 32)      0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 124, 124, 64)      18496     
_________________________________________________________________
batch_normalization_53 (Batc (None, 124, 124, 64)     

In [0]:
model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='acc',
                                factor=0.5,
                                patience=3,
                                min_lr=0.00001,
                                verbose=1)

In [8]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=20,
                    verbose=1,
                    callbacks=[reduce_lr])

Epoch 1/20


ResourceExhaustedError: ignored

In [0]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

In [0]:
predicted_class_indices=np.argmax(pred,axis=1)

In [0]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [0]:
filenames=test_generator.filenames
filenames=[f.split('.')[0] for f in filenames]

results=pd.DataFrame({"image_id":filenames,
                      "category":predictions})
results = results.sort_values(by = ['image_id'], ascending = [True])
results.to_csv("results-96x96-50-reduce_lr.csv",index=False)